# circRNA snakemake pipeline for Biodonostia

Authors: Leire Iparragirre, Inigo Prada and David Otaegi


I am going to develop a small snakemake pipeline for David Otaegui and Leire Iparraguirre at Biodonostia for analyzing circRNA sequencing data. The pipeline will have the following steps:

# Sequencing QC and Sequence Trimming

Nowadays, Illumina sequencing quality is good enough to skip this step. And it is not adviced any more to perform agresive sequence trimming on RNA-seq. Hence, I will include a simple sequencing quality checking step (fastqc-0.11.7) and lets trim afterwards If required ( I hope it is not :-)

# STAR index creation

   #### Overall explanation of this step

The pipeline will include a step were it is checked if STAR has its index of the genome and created if necessary. David and Leire, don't bother about this. An index is an efficient data structure to speed-up the alignment speed and reduce memory usage. If you are still interested on this (amazing) data structures, it is perfectly explained in this blog post:

http://blog.thegrandlocus.com/2016/07/a-tutorial-on-burrows-wheeler-indexing-methods

The reference data has been downloaded from Illumina iGenomes the 20-07-18 from this link https://support.illumina.com/sequencing/sequencing_software/igenome.html 

The Illumina iGenomes page includes a changelog, so If you wish to repeat the analysis be sure that you are working with the same files :)

ftp://ussd-ftp.illumina.com/CHANGE_LOG.txt

#### Technical explanation

 In this step we need to build a efficient data structure based on suffix arrays so that STAR can align million of reads per minute as efficient as possible. This is done with the following command:
 
 STAR --runThreadN {threads} --runMode genomeGenerate --outStd Log {log} --genomeDir {input.ref_genome_dir} --genomeFastaFiles {input.ref_genome}
 
 Where:
     - --runThreadN {threads} {Threads}: Number of threads to use to build the index (to increase speed)
     - --runMode genomeGenerate: Tell STAR to build the suffix array of the reference genome
     - --outStd Log {log}: Where to save log files
     - --genomeDir {input.ref_genome_dir}: Where to save the STAR index (This directory will contain) a bunch of files required  by STAR for aligning the read
     --genomeFastaFiles {input.ref_genome} Where is the Genome fasta file
     
     

# Alignment 

#### Overall explanation of this step

Once we have generate STAR index, we are ready to align the reads to reference genome. The general idea of this step is to determine to location of every read on the genome. To do so, we align the reads to the efficient representation of the genome we have created before. I completely skip all the alignment details, if you are interested on alignment algorithms, you can read about the alignment algorithms on chapter 2 (sections 2.1-2.5) of the following book:

Durbin, R., Eddy, S.R., Krogh, A. and Mitchison, G., 1998. Biological sequence analysis: probabilistic models of proteins and nucleic acids. Cambridge university press.

Once aligned, STAR outputs a bunch of files, been the most important one the bam file. The bam (binary alignment/map format ) is a file that contains crazy amount of information about how every read has align to the genome. Each read will contain encoded information as: alignment quality, is the alignment spliced?, are there any mismatches in the alignment? and a large etc.

You can read more about the sam format on the following links:

lightweight link:

https://genome.sph.umich.edu/wiki/SAM

heavyweight link:

https://samtools.github.io/hts-specs/SAMv1.pdf


#### Technical explanation 

In this step we align the reads to the genome with the following command


STAR --chimSegmentMin 10 --runThreadN {threads} --outSAMtype {params.bam_output} --genomeDir {params.reference} --readFilesCommand zcat --outFileNamePrefix {params.substr} --outStd Log {log} --readFilesIn {input.r1} {input.r2}

 - --chimSegementMin: Minimum length of chimeric reads (the ones indicating circRNA)
 - --runThreadN: Number of threads to use (speed up)
 - --genomeDir: Where is the index of the reference genome
 - --outFileNamePrefix: Output file name
 - --outStd Log: Where to put the log files
 - --readFilesCommand zcat: Assumes compressed input (tar.gz) 
 --readFilesIn: What are the input files

# Aligment QC

After the alignment one of the things that most scare biologist and bioinformaticians is the fact that reads align poorly or do not even align. To check so, we use the program samtools (v1.8 using htslib 1.8) which has severall tools to manage sam/bam files.

#### Technical explanation 

Command:


samtools flagstat input.bam

In this case, we will use samtools flagstat, which computes summary statistics of the alignment. The resulting samtools flagstat looks as the following:

20968800 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 duplicates
20968800 + 0 mapped (100.00%:nan%)
20968800 + 0 paired in sequencing
11431237 + 0 read1
9537563 + 0 read2
71098 + 0 properly paired (0.34%:nan%)
6633306 + 0 with itself and mate mapped
14335494 + 0 singletons (68.37%:nan%)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


Where:

    -Mapped: read considered as aligned
    - Paired in sequencing: If the reads were paired (paired-end sequencing)
    - read1: From the total (including unmmaped )reads, how many of them are the read 1 (forward read in paired end sequencing)
    - read2: From the total (including unmapped) reads, how many of them are the read 2  (reverse read )
    - properly paired: How many reads have align in the proper orientation (reverse and forward reads pointing at each other)
    - with itself and mate mapped: Both reads  (forward and reverse) mapped
    - Singletons: One of the paired reads is mapped and the other one is unmapped
    -with mate mapped to a different chr (mapQ>=5): Paired read were one of the reads has align to one chromosome and the other was is aligned to a different chromosome (this reads indicate, for example, interchromosomal translocations)
    -mapq>=5: Probability that the read is mapped wrongly aprox 0.31
    



    
# CircRNA detection


David and Leire, you are familiar with this step :)

The overall idea for this step is to detect those reads that overlap the backspliced junction of the circRNA.To the detect the circRNA we will use CIRCexplorer2 (v2.3.3).First, Circexplorer parses the chimeric junctions (which indicate circRNA ) from the STAR generated bam files, and, then, the circexplorer parsed file is annotated with the circRNA



#### Technical explanation 

Parse star output:

CIRCexplorer2 parse -t {params.aligner} -b {output.parsed_junctions} {input.junctions}

where:

-t: Which aligner I have use
-b: File name output
{input.junctions}: STAR backspliced junction output

Annotate STAR output:

CIRCexplorer2 annotate -r {input.ref_gene} -g {input.hg38} -b {input.parsed_junctions} -o {output.circrna} 

where:


-r: Gene annotation file of the reference genome
-g: REference genome fasta file
-b: Circexplorer parsed junctions
-o output file
    